In [1]:
import pandas as pd
import glob

### Import and concatenate all PM2.5 csv files into one

In [2]:
# Path to the directory containing CSV files 
csv_files_path = r"C:\Users\stlva\Documents\Python_Scripts\Data_analytics_bootcamp\air-quality-and-weather-analysis\Data\PM2.5\*.csv" # Adjust this path as necessary 

# Use glob to get all CSV file paths 
csv_files = glob.glob(csv_files_path) 

# Use list comprehension to read all CSV files and concatenate them into a single DataFrame 
PM2_5_df = pd.concat([pd.read_csv(file,low_memory=False) for file in csv_files], ignore_index=True) 

# Now PM2_5_df contains all the data from the 20 CSV files 


### Data cleaning and preprocessing

In [3]:
# Drop unwanted columns
PM2_5_df.drop(['Parameter Code', 
            'Sample Duration', 
            'Pollutant Standard', 
            'Event Type', 
            'Observation Count', 
            'Observation Percent',
            'Method Code', 
            'Method Name',  
            'Local Site Name', 
            'Address', 
            'Date of Last Change'],axis=1,inplace=True)

In [4]:
# Drop Canada
PM2_5_df.drop(PM2_5_df[(PM2_5_df['State Code']=='CC')].index,inplace=True)

# Change State Code dtype to int64
PM2_5_df['State Code'] = PM2_5_df['State Code'].astype('int64')

# Drop Mexico, Virgin Islands and Puerto Rico
PM2_5_df.drop(PM2_5_df[(PM2_5_df['State Code']==80)|(PM2_5_df['State Code']==78)|(PM2_5_df['State Code']==72)].index,inplace=True)

In [5]:
# Aggregating so we have 1 measurement per station per day per parameter
PM2_5_df = PM2_5_df.groupby(['State Code', 
                       'State Name', 
                       'County Code', 
                       'County Name', 
                       'City Name', 
                       'Site Num', 
                       'Datum', 
                       'Latitude', 
                       'Longitude', 
                       'Date Local', 
                       'Parameter Name', 
                       'Units of Measure']).agg({'Arithmetic Mean':'mean', '1st Max Value':'max','1st Max Hour':'max','CBSA Name':'first','AQI':'mean'}).reset_index()

In [6]:
# Create unique identifier for each measurement
PM2_5_df['measurement_id'] = PM2_5_df['State Code'].apply(str) + '_' + PM2_5_df['County Code'].apply(str) + '_' + PM2_5_df['Site Num'].apply(str) + '_' + PM2_5_df['Date Local'].str.replace('-','')

In [7]:
# checking for the parameters included in the dataframe
PM2_5_df['Parameter Name'].unique()

array(['PM2.5 - Local Conditions'], dtype=object)

In [8]:
# Renaming column to match the value measured and droping unnecessary columns
PM2_5_df.rename(columns={'Arithmetic Mean':'PM2_5(µg/m³ LC)','1st Max Value':'PM2_5(µg/m³ LC) Max Value', '1st Max Hour':'PM2_5(µg/m³ LC) Max Hour'},inplace=True)
PM2_5_df.drop(['Parameter Name', 'Units of Measure'],axis=1,inplace=True)

In [9]:
# Convert date column to datetime
PM2_5_df['Date Local'] = pd.to_datetime(PM2_5_df['Date Local'])

In [11]:
PM2_5_df.head()

,State Code,State Name,County Code,County Name,City Name,Site Num,Datum,Latitude,Longitude,Date Local,PM2_5(µg/m³ LC),PM2_5(µg/m³ LC) Max Value,PM2_5(µg/m³ LC) Max Hour,CBSA Name,AQI,measurement_id
0,1,Alabama,3,Baldwin,Fairhope,10,NAD83,30.497478,-87.880258,2004-01-01,18.5,18.5,0,"Daphne-Fairhope-Foley, AL",64.0,1_3_10_20040101
1,1,Alabama,3,Baldwin,Fairhope,10,NAD83,30.497478,-87.880258,2004-01-04,4.0,4.0,0,"Daphne-Fairhope-Foley, AL",17.0,1_3_10_20040104
2,1,Alabama,3,Baldwin,Fairhope,10,NAD83,30.497478,-87.880258,2004-01-07,5.6,5.6,0,"Daphne-Fairhope-Foley, AL",23.0,1_3_10_20040107
3,1,Alabama,3,Baldwin,Fairhope,10,NAD83,30.497478,-87.880258,2004-01-10,16.4,16.4,0,"Daphne-Fairhope-Foley, AL",60.0,1_3_10_20040110
4,1,Alabama,3,Baldwin,Fairhope,10,NAD83,30.497478,-87.880258,2004-01-13,16.4,16.4,0,"Daphne-Fairhope-Foley, AL",60.0,1_3_10_20040113


In [12]:
#Saving final df to csv
#PM2_5_df.to_csv(r"C:\Users\stlva\Documents\Python_Scripts\Data_analytics_bootcamp\air-quality-and-weather-analysis\Cleaning\all_PM2_5_clean.csv",index=False)
PM2_5_df.to_csv(r"C:\Users\stlva\Google Drive\Big blue data academy\main_branch\concatenated data\all_PM2_5_clean.csv",index=False)